In [1]:
import cv2
import numpy as np
import math
import sympy as sym
import matplotlib.pyplot as plt
from scipy import signal as sig

In [2]:
class Point:
    RADIUS = 5  # Global radius for all points

    def __init__(self, x_or_iterable, y=None):
        if isinstance(x_or_iterable, tuple):
            x, y = x_or_iterable
        elif hasattr(x_or_iterable, '__iter__'):
            x, y = x_or_iterable
        else:
            x, y = x_or_iterable, y
        self.x = int(x)
        self.y = int(y)
    def __iter__(self):
        return iter(self.__dict__.values())
    def draw(self, img, color=(255, 255, 255), thickness=-1):
        cv2.circle(img, (self.x, self.y), Point.RADIUS, color, thickness)

class Vector:
    def __init__(self, start, end, thickness=2, colormap=None,idx=0):
        self.start = start
        self.end = end
        self.thickness = thickness
        self.colormap = colormap
        self.idx = idx

    def draw(self, img):
        if self.colormap is None:
            color = (0, 255, 0)
        else:
            color = self.colormap(self.idx)
        cv2.arrowedLine(img, (self.start.x, self.start.y), (self.end.x, self.end.y), color, self.thickness)

def num_points_colormap(num_points,max_num_points=10):
    """
    A colormap that maps the number of points to a color.
    Fewer points are blue, more points are red.
    """
    
    r = min(255, int(num_points / max_num_points * 255))
    b = max(0, 255 - r)
    return (b, 0, r)

In [3]:
def motorPlant(velocities:tuple,ref:tuple,sampleRate = 1)->tuple:
    xdist,ydist = ref
    distance_between_motors = 10 #cm
    vl,vr = velocities
    phi = (vr-vl)/distance_between_motors
    # Calculate forward velocity
    v = (vl + vr) 
    print(v)
    xdot =v * math.sin(phi *sampleRate / 2) 
    ydot =v * math.cos(phi * sampleRate/ 2) 
    print(f"")
    xbar =  xdist - xdot * sampleRate #  new x
    ybar = ydist - ydot * sampleRate #new y
    print(f"x:{xdist} y:{ydist}|Xdot: {xdot} Ydot:{ydot} |xbar : {xbar} ybar:{ybar}")
    return (xbar,ybar)


In [12]:
def motorPlant_tf():
    # Define Laplace variable
    s = sym.symbols('s')

    # Define transfer function variables
    vl, vr, xdist, ydist = sym.symbols('vl vr xdist ydist', real=True, positive=True)

    # Define system parameters
    sampleRate = 1
    distance_between_motors = 10  # cm

    # Calculate angular velocity
    phi = (vr - vl) / distance_between_motors

    # Calculate forward velocity
    v = (vl + vr)

    # Calculate xdot and ydot
    xdot = v * sym.sin(phi * sampleRate / 2)
    ydot = v * sym.cos(phi * sampleRate / 2)

    # Calculate xbar and ybar
    xbar = xdist - xdot / s
    ybar = ydist - ydot / s

    # Define transfer function
    num = [1, 0]
    den = [1/s, 1/s]
    tf = sig.TransferFunction(num, den)
    return tf


In [31]:
SCALE = 2
IMG_WIDTH = 300 * SCALE
IMG_HEIGHT = 300 *SCALE
XDIST = -10 *SCALE
YDIST = 110 *SCALE
start = (XDIST, YDIST)
RATIO = 0.25
SPEED = 1.5
SAMPLE_RATE =10
stepSize = (SPEED*(1-RATIO)*SCALE,SPEED*RATIO*SCALE)
# Create a black image
img = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), np.uint8)
# Define points as Point objects
centre = Point(IMG_HEIGHT//2, IMG_WIDTH//2)
p2 = Point(centre.x + XDIST, centre.y - YDIST)
# motorPlant((1,1),p2)
x,y = motorPlant(stepSize, start,sampleRate=SAMPLE_RATE)
newPoint = Point(centre.x+x,centre.y-y)
points = [centre,p2,newPoint]
vectors = [Vector(centre, p2, colormap=num_points_colormap, thickness=2,idx=0),
           Vector(centre,newPoint,colormap=num_points_colormap, thickness=2,idx=1)]

for i in range(2,10):
    x,y = motorPlant(stepSize, (x,y),sampleRate=SAMPLE_RATE)
    newPoint = Point(centre.x+x,centre.y-y)
    points.append(newPoint)
    vectors.append(Vector(centre, newPoint, colormap=num_points_colormap, thickness=2,idx=i))


    # Draw the points and vectors on the image
    for point in points:
        point.draw(img)

    for vector in vectors:
        vector.draw(img)

    # Display the image
    cv2.imshow('Points and Vectors', img)
cv2.waitKey(0)
cv2.destroyAllWindows()



3.0

x:-20 y:220|Xdot: -2.0449162800700025 Ydot:2.1950666066214626 |xbar : 0.449162800700023 ybar:198.04933393378536
3.0

x:0.449162800700023 y:198.04933393378536|Xdot: -2.0449162800700025 Ydot:2.1950666066214626 |xbar : 20.898325601400046 ybar:176.09866786757073
3.0

x:20.898325601400046 y:176.09866786757073|Xdot: -2.0449162800700025 Ydot:2.1950666066214626 |xbar : 41.34748840210007 ybar:154.1480018013561
3.0

x:41.34748840210007 y:154.1480018013561|Xdot: -2.0449162800700025 Ydot:2.1950666066214626 |xbar : 61.79665120280009 ybar:132.19733573514145
3.0

x:61.79665120280009 y:132.19733573514145|Xdot: -2.0449162800700025 Ydot:2.1950666066214626 |xbar : 82.24581400350012 ybar:110.24666966892683
3.0

x:82.24581400350012 y:110.24666966892683|Xdot: -2.0449162800700025 Ydot:2.1950666066214626 |xbar : 102.69497680420014 ybar:88.2960036027122
3.0

x:102.69497680420014 y:88.2960036027122|Xdot: -2.0449162800700025 Ydot:2.1950666066214626 |xbar : 123.14413960490016 ybar:66.34533753649758
3.0

x:12

In [17]:
def step_response(tf, T_end=10):
    # Generate time vector
    t = np.linspace(0, T_end, 1000)

    # Calculate step response
    _, y = signal.step(tf, T=t)

    # Plot step response
    plt.plot(t, y)
    plt.xlabel('Time (s)')
    plt.ylabel('Output')
    plt.title('Step response')
    plt.grid()
    plt.show()

    # Return time and output vectors
    return t, y
step_response(motorPlant_tf())

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''